In [ ]:
import os
import serial
import serial.tools.list_ports
import threading
import time
import datetime
from datetime import datetime, timedelta
import pandas as pd
import glob
from joblib import load
import numpy as np

MONITOR_DIR = r'C:\Users\77156\Desktop\Raman\SNV'  # 监控目录
CSV_PATTERN = '*.csv'  # CSV文件的模式
MODEL = load('model.pkl')  # 实例化模型


def snv_transformation(spectrum):
    """
    进行标准正态变量变换（SNV）处理拉曼光谱数据。

    参数:
    spectrum -- 一维NumPy数组，代表拉曼光谱数据。

    返回:
    snv_spectrum -- 经过SNV变换后的拉曼光谱数据。
    """
    spectrum_mean = np.mean(spectrum)
    spectrum_std = np.std(spectrum)

    # 避免除以零
    if spectrum_std == 0:
        spectrum_std = 1.0

    snv_spectrum = (spectrum - spectrum_mean) / spectrum_std
    return snv_spectrum

def process_raman_csv(input_file_path, output_file_path):
    """
    读取CSV文件，对拉曼光谱数据应用SNV变换，并将原始波数和变换后的光谱强度保存到新的CSV文件。

    参数:
    input_file_path -- 输入CSV文件的路径。
    output_file_path -- 输出CSV文件的路径。
    """
    # 读取CSV文件
    df = pd.read_csv(input_file_path)

    # 假设CSV的第一列是波数，第二列是光谱强度数据
    raman_shift = df.iloc[:, 0]  # 波数
    spectrum = df.iloc[:, 1].values  # 光谱强度数据

    # 对光谱数据进行SNV变换
    snv_spectrum = snv_transformation(spectrum)

    # 创建一个新的DataFrame，包含原始波数和变换后的光谱强度
    snv_df = pd.DataFrame({
        'Raman Shift': raman_shift,  # 假设原始CSV文件中波数列的列名是'Raman Shift'
        'SNV Intensity': snv_spectrum  # 新的列名，代表SNV变换后的光谱强度
    })

    # 保存新的DataFrame到CSV文件
    output_file_path = output_file_path.replace('.csv', '_snv.csv')
    snv_df.to_csv(output_file_path, index=False)

def get_latest_csv(folder_path):
    """
    获取指定文件夹下最新的CSV文件路径。

    参数:
    folder_path -- 包含CSV文件的文件夹路径。

    返回:
    最新CSV文件的路径。
    """
    # 获取文件夹内所有的文件和目录名称
    files_and_dirs = os.listdir(folder_path)

    # 使用glob过滤出所有的CSV文件
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    # 找到最新的文件，基于文件的修改时间
    latest_csv = max(csv_files, key=os.path.getmtime)

    return latest_csv


def read_and_preprocess_csv(file_path):
    df = pd.read_csv(file_path)
    data = df.iloc[:, 1].values[1:3252]  # 假设第一列是拉曼光谱特征值
    index_list = np.loadtxt('index_list.txt')
    
    values = []
    # 遍历索引列表，使用iloc根据索引位置选取对应的第二列的数据
    for index in index_list:
        if index <= len(df):  # 确保索引不会超出DataFrame的行数
            value = df.loc[df['Raman Shift'] == index, 'SNV Intensity'].values
            values.append(value)
        else:
            values.append(None)  # 如果索引超出范围，添加None

    # 打印结果
    #for idx, val in enumerate(values):
        #print(f"The value corresponding to index {index_list[idx]} is: {val}")
    processed_data = np.array(values)
    return processed_data

# 根据预测浓度计算补料时间
def calculate_feed_time(predicted_concentration, threshold):
    if predicted_concentration < threshold:
        feed_time = ((2 - predicted_concentration) * 2.5) / 0.35
    else:
        feed_time = 0  # 如果葡萄糖浓度不低，补料时间为0
    return feed_time

def predict_glucose_concentration(file_paths, threshold):
    predictions = []
    feed_times = []
    for file_path in file_paths:
        processed_data = read_and_preprocess_csv(file_path)
        prediction = MODEL.predict(processed_data.reshape(1, -1))  # 确保数据是2D数组
        # 修正：检查prediction是否为一维数组，然后获取浓度值
        if isinstance(prediction, np.ndarray) and prediction.ndim == 1:
            concentration_value = prediction[0]
            if concentration_value < 0:
                concentration_value = 0
        else:
            concentration_value = prediction
            if concentration_value < 0:
                concentration_value = 0
        
        predictions.append(concentration_value)  # 假设模型返回的是单个浓度值
        
    
    # 计算均值
    mean_concentration = sum(predictions) / len(predictions)
    feed_time = calculate_feed_time(concentration_value, threshold)
    feed_times.append(feed_time)
    control_pump(concentration_value, threshold, feed_time)  # 控制泵
    return predictions, mean_concentration, feed_times

# 添加一个新的函数来控制泵
def control_pump(glucose_concentration, threshold, feed_time):
    ser = serial.Serial('COM2', 1200, timeout=0.1, parity='E') # 确保端口名称正确
    try:
        if glucose_concentration < threshold:
            # 如果葡萄糖浓度低于阈值，启动泵
            cmd1 = bytes.fromhex('E9 01 06 57 4A 01 F4 01 01 EF')
            ser.write(cmd1)
            # 等待补料时间
            time.sleep(feed_time)
            # 停止泵
            cmd2 = bytes.fromhex('E9 01 06 57 4A 00 00 00 01 1B')
            ser.write(cmd2)
        else:
            # 如果葡萄糖浓度高于或等于阈值，不启动泵
            pass
    finally:
        ser.close()

def save_concentration_mean(file_paths, mean_concentration):
    base_name = os.path.basename(os.path.commonprefix(file_paths))  # 假设文件名有共同的前缀
    output_dir = r'C:\Users\77156\Desktop\feedback\predict_data'
    save_path = os.path.join(output_dir, f"{base_name}_mean_concentration.csv")
    # 检查文件是否存在，如果不存在，先以写入模式打开并写入标题行
    if not os.path.exists(save_path):
        with open(save_path, 'w', newline='') as f:
            f.write('Mean Glucose Concentration,File Path\n')
    
    # 以追加模式打开文件，将数据追加到文件末尾
    with open(save_path, 'a', newline='') as f:
        f.write(f"{mean_concentration}, {os.path.basename(file_paths[0])}\n")  # 使用文件名而非完整路径

def check_and_process_files(threshold):
    # 获取最新的五个CSV文件
    file_paths = sorted(glob.glob(os.path.join(MONITOR_DIR, CSV_PATTERN)), reverse=True)[:5]

    # 检查是否有文件，如果有则进行处理
    if file_paths:
        # 预测葡萄糖浓度
        predictions, mean_concentration, feed_times = predict_glucose_concentration(file_paths, threshold)

        # 保存均值
        save_concentration_mean(file_paths, mean_concentration)

        # 输出葡萄糖浓度均值
        print(f"Mean Glucose Concentration: {mean_concentration}")

def process_latest_five_csv(folder_path, output_folder):
    # 获取最新的5个CSV文件
    csv_files = sorted(glob.glob(os.path.join(folder_path, "*.csv")), key=os.path.getmtime, reverse=True)[:5]
    
    # 处理这5个文件
    for csv_file in csv_files:
        # 构造输出文件的路径
        base_name = os.path.basename(csv_file)
        output_file = os.path.join(output_folder, f"processed_{base_name}")
        process_raman_csv(csv_file, output_file)
        print(f"处理完成：{csv_file} -> {output_file}")

def job():
    # 设置文件夹路径
    folder_path = r'C:\Users\77156\Desktop\Raman\data'
    output_folder = r'C:\Users\77156\Desktop\Raman\SNV'
    
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 执行处理最新的5个CSV文件
    process_latest_five_csv(folder_path, output_folder)

# 设置定时任务，每半小时执行一次
#schedule.every(30).minutes.do(job)

#print("定时任务已启动，每半小时处理最新的5个CSV文件。")

        
def should_run():
    """
    判断是否应该运行文件处理逻辑。
    这里可以根据你的具体需求来设置条件，例如只在工作日运行等。
    """
    # 这里设置为始终返回True，表示每天都运行
    return True
        
def run_for_ten_days(threshold):
    start_time = datetime.now()
    end_time = start_time + timedelta(days=10)
    
    while datetime.now() < end_time:
        if should_run():
            job()
            check_and_process_files(threshold)
        
        # 暂停30分钟
        time.sleep(1800)

if __name__ == "__main__":
    
    glucose_threshold = 2  # 假设2 g/L是葡萄糖浓度的阈值
    # 启动连续监测
    run_for_ten_days(glucose_threshold)